In [5]:
openai_api_key = os.getenv("OPENAI_API_KEY")


In [2]:
!pip install python-dotenv

In [1]:
pip install tiktoken


   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/893.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/893.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/893.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/893.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/893.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/893.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/893.9 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/893.9 kB ? eta -:--:--
   ---------------------- --------------- 524.3/893.9 kB 158.3 kB/s eta 0:00:03
   ---------------------- --------------- 524.3/893.9 kB 158.3 kB/s eta 0:00:03
   ---------------------- --------------- 524.3/893.9 kB 158.3 kB/s eta 0:00:03
   ---------------------- --------------- 524.3/893.9 kB 158.3 kB/s eta 0:00:03
   ----------------------

In [1]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install sentence-transformers


  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.53.3-py3-none-any.whl.metadata (40 kB)
  Using cached torch-2.7.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached huggingface_hub-0.33.4-py3-none-any.whl.metadata (14 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached tokenizers-0.21.2-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl.metadata (12 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached sentence_transformers-5.0.0-py3-none-any.whl (470 kB)
Using cached transformers-4.53.3-py3-none-any.whl (10.8 MB)
Using cached huggingface_hub-0.33.4-py3-none-any.whl 

In [1]:
pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 1.9 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 3.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.agents import Tool, initialize_agent
from langchain.llms import OpenAI
from langchain.chains import LLMMathChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents.agent_types import AgentType
from langchain.embeddings import HuggingFaceEmbeddings


load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set")



llm = OpenAI(temperature=0)

# Load and split PDF
def load_pdf(path):
    loader = PyPDFLoader(path)
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_documents(documents)

# Build vector DB
def create_retriever(docs):
    embeddings = HuggingFaceEmbeddings()
    db = FAISS.from_documents(docs, embeddings)
    return db.as_retriever()

# Set up QA tool
def build_tools(pdf_path):
    try:
        docs = load_pdf(pdf_path)
        retriever = create_retriever(docs)
        qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
        pdf_tool = Tool(name="PDF_QA", func=qa.run, description="Answers questions from uploaded PDF")

        math_chain = LLMMathChain(llm=llm, verbose=True)
        calculator_tool = Tool(name="Calculator", func=math_chain.run, description="Performs math")

        return [pdf_tool, calculator_tool]
    except Exception as e:
        print("Tool creation error:", e)
        return []

# Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

def get_agent(tools):
    return initialize_agent(
        tools,
        llm=llm,
        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True
    )

if __name__ == "__main__":
    tools =build_tools(r"C:\Users\Asus\dictionary.pdf")  
    agent = get_agent(tools)

    question = input("What is this document about?")
    response = agent.run(question)
    print("🤖 Agent:", response)


C:\Users\Asus\AppData\Local\Temp\ipykernel_13800\1801428262.py:24: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)
C:\Users\Asus\AppData\Local\Temp\ipykernel_13800\1801428262.py:56: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_13800\1801428262.py:35: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should

Tool creation error: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load your data
df = pd.read_csv("D:\DATA SCIENCE LEARNING\STRUCTURED AND UNSTRUCTURED DATA\house.csv")

# Inspect and choose features/target
print(df.columns)
X = df[['area_sqft', 'bedrooms']]  
y = df['price']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("R² Score:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

Index(['price', 'area_sqft', 'bedrooms', 'bathrooms', 'year_built', 'location',
       'distance_to_city_center', 'has_garage', 'has_pool', 'condition',
       'crime_rate', 'school_rating'],
      dtype='object')
R² Score: 0.9801091757799386
MSE: 652402873.1233342
